#Atividade 20 - Migração e Integração de Bases de Dados

A SoulCode Academy utiliza dois sistemas distintos para cadastro de vendas, um em sua matriz e outro em suas filiais. O sistema da matriz utiliza um db noSQL enquanto os das filiais ainda utilizam um SQL. As tabelas a seguir são amostras de dados dos dois tipos de db enviadas para auxiliar seu desenvolvimento. Crie uma db em SQL e uma noSQL, e adicione essas amostrar para criar um ambiente simulando o problema real. A seguir crie um código em Python que leia a db SQL, corrija, padronize e insira os dados na db noSQL.


Devem ser entregues:
- os códigos desenvolvidos
- um cronograma apresentando quem fez o que e quando
- uma apresentação em slides apresentando o desenvolvimento criado

##INSTALLS E BIBLIOTECAS

In [ ]:
# Conexão com o google drive, onde os conectores do postgres e mongo se encontram
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Pandas é uma biblioteca de manipulação de dados
# Panderas será utilizado para verificação de dados
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
!pip install pymongo[srv]

In [ ]:
# pymongo é uma biblioteca que será utilizado para configurar a conexão com um banco de dados noSQL
from pymongo import MongoClient
# Função que retorna as configuração do banco de dados Mongo utilizados neste exercício
!cp /content/drive/MyDrive/SoulCode/modulos/mongo.py .
from mongo import get_mongo_access

In [ ]:
# Acesso à Classe que utiliza a biblioteca psycopg2 para acessar bancos de dados postgres
# Também contem função que retorna as configurações do banco de dados postgres utilizados neste exercício
!cp /content/drive/MyDrive/SoulCode/modulos/postgres.py .
from postgres import Conector_postgres, get_postgres_access

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


##CONEXÕES

### Conexão com postgres

In [ ]:
# Configuração de conexão
pg_host, pg_db, pg_user, pg_pwd = get_postgres_access()
banco_pg = Conector_postgres(pg_host,pg_db, pg_user,pg_pwd)
colunas = ['nota_fiscal','vendedor','total']

In [ ]:
# Download
dados = banco_pg.selecionar('select * from sistema_a')
df_pg = pd.DataFrame(dados, columns = colunas)

In [ ]:
df_pg

### Conexão com mongo

In [ ]:
# Obtendo o IP do notebook para liberar conexão no banco de dados Mongo
!curl ipecho.net/plain

34.80.249.145

In [ ]:
# Obtendo a string de conexão, nome do banco de dados e nome da collection
conn, db_name, collection_name = get_mongo_access()

# Estabelecendo a conexão com o Mongo Atlas
client = MongoClient(conn)

# Definindo o banco de dados que será utilizado
db = client[db_name]

# Definindo a coleção que será utilizada
collection = db[collection_name]

In [ ]:
# Obtendo os dados o banco
df_m = pd.DataFrame(list(collection.find()))

In [ ]:
df_m

## TRATAMENTO DE DADOS

In [ ]:
# Criando backup
df_pg2 = df_pg.copy()
df_m2 = df_m.copy()

### Tratando postgres

In [ ]:
# Observação superficial do conteúdo
df_pg2

In [ ]:
# Analisando os tipos de dados de cada coluna
df_pg2.dtypes

In [ ]:
# Procurando por valores nulos
df_pg2.isnull().sum()

In [ ]:
# Substituindo os valores nulos
df_pg2.fillna('não informado', inplace=True)

In [ ]:
# Analisando a coluna nota_fiscal
df_pg2['nota_fiscal'].is_unique

In [ ]:
# Convertendo os valores da coluna 'total' para o tipo float
df_pg2['total'] = pd.to_numeric(df_pg2['total'])

In [ ]:
# Procurando por caracteres estranhos
sorted(pd.unique(df_pg2['total']))

In [ ]:
sorted(pd.unique(df_pg2['vendedores']))

### Tratando mongo

In [ ]:
# Análise superficial
df_m2.head()

In [ ]:
# Procurando por valores nulos
df_m2.isnull().sum()

In [ ]:
# Analisando os tipos de dados de cada coluna
df_m2.dtypes

In [ ]:
# Substituindo os valores nulos
df_m2.fillna('não informado', inplace=True)

In [ ]:
# Analisando a coluna nota_fiscal
df_m2['nota_fiscal'].is_unique

In [ ]:
# Procurando por caracteres estranhos
sorted(pd.unique(df_m2['total']))

In [ ]:
sorted(pd.unique(df_m2['vendedores']))

### Concatenando os Dataframes

In [ ]:
df = pd.concat([df_pg2,df_m2], ignore_index=True)

## UPLOAD PARA O MONGO

In [ ]:
df_dict = df.to_dict('records')

# Criando a collection que receberá os dados consolidados
db.create_collection('venda')
collection_in = db['venda']

# Inserindo os dados no mongodb
collection_in.insert_many(df_dict)


## LOG



20/05 <br>
14:45 Tabela atividade20 criada no postgres localhost <br>
15:52 Collection atv20_bruto criada no Mongodb Atlas <br>
20:16 Conversando com colegas de turma decidi utilizar um banco de dados postgres na nuvem para melhor simular o propósito do exercício<br>
21/05<br>
14:10 Criado um banco de dados postgres no Heroku<br>
15:00 Criada a tabela sistema_a e importando os dados necessários através do uso do pgadmin<br>
20:00 Criando a apresentação no canvas